# train

> Main training loop

In [ ]:
import os
import subprocess

from dotenv import load_dotenv
import gymnasium as gym
import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig
from omegaconf import OmegaConf
import wandb

from rlmm.core import *
from rlmm.utils import *

In [ ]:
@hydra.main(version_base=None, config_name="conf", config_path="../conf")
def train(cfg: DictConfig) -> None:
    load_dotenv()

    cfg_dict: dict = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)  # type: ignore

    log_fn = None
    run_id = None
    if cfg["wandb"]:
        wandb.init(
            config=cfg_dict,
            project=os.environ["WANDB_PROJECT"],
            entity=os.environ["WANDB_ENTITY"],
            save_code=True,
        )
        wandb.config.update(
            {
                "git_branch": subprocess.check_output(
                    "git rev-parse --abbrev-ref HEAD",
                    shell=True,
                    universal_newlines=True,
                ).strip(),
                "git_hash": subprocess.check_output(
                    "git rev-parse HEAD", shell=True, universal_newlines=True
                ).strip(),
            }
        )
        wandb.run.log_code(".")

        log_fn = wandb.log
        run_id = wandb.run.id

    set_seed(cfg["seed"])

    trainer = instantiate(cfg["trainer"])
    trainer = trainer(log_fn=log_fn, run_id=run_id)
    env_fn = lambda: instantiate(cfg["env_train"])
    env = gym.vector.AsyncVectorEnv([env_fn for _ in range(cfg["num_envs"])])
    agent = instantiate(cfg["agent"])

    if cfg["dry_run"]:
        print("")
        return

    trainer.train(env, agent, cfg["rounds_train"])

    if cfg["eval"]:
        print("Evaluating...")
        env = instantiate(cfg["env_eval"])
        trainer.evaluate(env, agent, cfg["rounds_eval"])

In [ ]:
if __name__ == "__main__":
    train()